In [1]:
import numpy as np
from scipy.interpolate import RectBivariateSpline as RBS
from scipy.io import savemat
from ipynb.fs.defs.eigen import eigen_C, eigen_S

In [2]:
def _tensorfield(X, Y, A, type = 'LCS'):
    
    # compute eigenvalues lambda_1, lambda_2
    lambda1 = np.zeros((A.shape[0], A.shape[1]))
    lambda2 = lambda1.copy()
    eigenv1 = np.zeros((A.shape[0], A.shape[1], 2))
    eigenv2 = eigenv1.copy()
    
    for i in range(X.shape[0]):
        
        for j in range(Y.shape[1]):
            
            if type == 'LCS':
            
                lambda1[i,j], lambda2[i,j], eigenv1[i,j,:], eigenv2[i,j,:] = eigen_C(A[i,j,:,:])
                
                if np.isfinite(lambda1[i,j]):
                    T = np.array([eigenv1[i,j,:], eigenv2[i,j,:]]).T
                    LAMBDA = np.diag([lambda1[i,j], lambda2[i,j]])
                    A[i, j,:, :] = T.T@LAMBDA@T
                
            elif type == 'OECS':
                
                lambda1[i,j], lambda2[i,j], eigenv1[i,j,:], eigenv2[i,j,:] = eigen_S(A[i,j,:,:])
                
                if np.isfinite(lambda1[i,j]):
                    T = np.array([eigenv1[i,j,:], eigenv2[i,j,:]]).T
                    LAMBDA = np.diag([lambda1[i,j], lambda2[i,j]])
                    A[i, j,:, :] = T.T@LAMBDA@T
                
            else:
                
                sys.exit("type must be either LCS or OECS")
    
    A11 = np.nan_to_num(A[:,:,0,0], nan=0.0)
    A12 = np.nan_to_num(A[:,:,0,1], nan=0.0)
    A22 = np.nan_to_num(A[:,:,1,1], nan=0.0)
    
    interp_A11 = RBS(Y[:,0], X[0,:], A11)
    interp_A12 = RBS(Y[:,0], X[0,:], A12)
    interp_A22 = RBS(Y[:,0], X[0,:], A22)
    
    interp_lambda1 = RBS(Y[:,0], X[0,:], lambda1)
    interp_lambda2 = RBS(Y[:,0], X[0,:], lambda2)
    
    rho_x = (X[0,1]-X[0,0])/20
    rho_y = (Y[1,0]-Y[0,0])/20
    
    A11x = np.zeros((A.shape[0], A.shape[1]))*np.nan
    A11y = A11x.copy()
    A12x = A11x.copy()
    A12y = A11x.copy()
    A22x = A11x.copy()
    A22y = A11x.copy()
    
    lam1x = A11x.copy()
    lam1y = A11y.copy()
    lam2x = A11x.copy()
    lam2y = A11x.copy()
    
    for i in range(X.shape[0]):
        
        for j in range(Y.shape[1]):
            
            x = [X[i,j], Y[i, j]]
            
            A11x[i, j] = (interp_A11(x[1], x[0]+rho_x)[0][0]-interp_A11(x[1], x[0]-rho_x)[0][0])/(2*rho_x)
            A11y[i, j] = (interp_A11(x[1]+rho_y, x[0])[0][0]-interp_A11(x[1]-rho_y, x[0])[0][0])/(2*rho_y)
            A12x[i, j] = (interp_A12(x[1], x[0]+rho_x)[0][0]-interp_A12(x[1], x[0]-rho_x)[0][0])/(2*rho_x)
            A12y[i, j] = (interp_A12(x[1]+rho_y, x[0])[0][0]-interp_A12(x[1]-rho_y, x[0])[0][0])/(2*rho_y)
            A22x[i, j] = (interp_A22(x[1], x[0]+rho_x)[0][0]-interp_A22(x[1], x[0]-rho_x)[0][0])/(2*rho_x)
            A22y[i, j] = (interp_A22(x[1]+rho_y, x[0])[0][0]-interp_A22(x[1]-rho_y, x[0])[0][0])/(2*rho_y)
            
            lam1x[i, j] = (interp_lambda1(x[1], x[0]+rho_x)[0][0]-interp_lambda1(x[1], x[0]-rho_x)[0][0])/(2*rho_x)
            lam1y[i, j] = (interp_lambda1(x[1]+rho_y, x[0])[0][0]-interp_lambda1(x[1]-rho_y, x[0])[0][0])/(2*rho_y)
            lam2x[i, j] = (interp_lambda2(x[1], x[0]+rho_x)[0][0]-interp_lambda2(x[1], x[0]-rho_x)[0][0])/(2*rho_x)
            lam2y[i, j] = (interp_lambda2(x[1]+rho_y, x[0])[0][0]-interp_lambda2(x[1]-rho_y, x[0])[0][0])/(2*rho_y)    
    
    return X, Y, lambda1, lambda2, eigenv1, eigenv2, A11, A12, A22, A11x, A11y, A12x, A12y, A22x, A22y